# Motion Vector Generator datapath generator

#### Parameters

In [18]:
import math 
import numpy as np
from datetime import datetime
from IPython.display import Javascript

In [19]:
####################################################
# Inputs definitions
####################################################
INPUT_N = 4                                     # Input size is a INPUT_N x INPUT_N block of 4x4 sub-blocks
INPUT_COORD_WIDTH = 8                           # Number of bits as input for each block in an input line
INPUT_MACROBLOCK_SIZE = 16                      # Number of pixels in one macroblock (16 = 4x4)
PIXEL_WIDTH = 8                                 # Number of bits for each integer pixel from the reference frame


####################################################
# Interpolator definitions
####################################################
INPUT_LINE_SIZE = 9                             # Input size for ONE line loaded to the buffer (9 integer pixels)
TAPS = 6                                        # Number of taps
FILTERS = 15                                    # Number of filters (= number of outputs expected)
OUT_1ST_INTERP_WIDTH = 11                       # Width for every 15 interpolation output for FIRST PASS
OUT_2ND_INTERP_WIDTH = 14                       # Width for every 15 interpolation output for SECOND PASS
BUFFER_LINES = 9                                # Number of lines in the transposed buffer
BUFFER_COLUMNS = 4                              # Number of columns in the transposed buffer


####################################################
# Motion Vector Generator definitions
####################################################
INPUT_COORD_WIDTH = 8                           # Input size for the coordinates X and Y
INPUT_CPMV_WIDTH = 8                            # Input size for the control point motion vectors (CPMV0 and CPMV1)
OUTPUT_MV_GEN_WIDTH = 19                        # Output size of the Motion Vector module (4 and 6 parameters are equal)


#### Intermediate variables calculation

In [20]:
####################################################
# MV GENERATOR DATAPATH DEFINITIONS
####################################################
MV_GEN_FILE_NAME = 'MV_gen_datapath'            # Interpolation datapath file name
DATE_MODIF = str(datetime.today().strftime('%d/%m/%Y'))

# Control inputs 
# Template: [signal name, bit width, io type, description]
# Obs: io type is relative to this datapath pov
MV_GEN_CTRL_SIGNALS = [
    ['CLK', 1, 'input', ''],
    ['RST_ASYNC_MV_GEN', 1, 'input', '\t\t\t\t\t// Set of reg resets: coord_X, coord_Y, X, Y, MV_0, MV_1, gen_MV_X, gen_MV_Y and count_block.'], 
    # ['rst_async_reg_count_block', 1, 'input', ],
    ['WRITE_REGS_COORDS', 1, 'input', '\t\t\t\t// Enable writing to registers coord_X and coord_Y.'], 
    ['WRITE_REGS_CPMVS', 1, 'input', '\t\t\t\t\t// Enable writing to registers MV_0 and MV_1.'], 
    ['WRITE_REGS_GEN_MVS', 1, 'input', '\t\t\t// Enable writing to registers gen_MV_X and gen_MV_Y.'], 
    ['WRITE_REG_X', 1, 'input', '\t\t\t\t\t// Enable writing for reg_X.'], 
    ['WRITE_REG_Y', 1, 'input', '\t\t\t\t\t// Enable writing for reg_Y.'], 
    ['WRITE_REG_COUNT_BLOCK', 1, 'input', '\t\t\t// Enable writing for reg_count_block.'], 
    ['SEL_X', 1, 'input', '\t\t\t\t\t// Select: 0 - original X coord; 1: incremented X coord.'], 
    ['SEL_Y', 1, 'input', '\t\t\t\t\t// Select: 0 - original Y coord; 1: incremented Y coord.'], 
    ['COORD_X', INPUT_COORD_WIDTH, 'input signed', '\t\t\t// First X coordenate.'],
    ['COORD_Y', INPUT_COORD_WIDTH, 'input signed', '\t\t\t// First Y coordenate.'],
    ['CPMV_0', 2*INPUT_CPMV_WIDTH, 'input signed', '\t\t\t// Motion vector 0. MSB = x component, LSB = y component.'],
    ['CPMV_1', 2*INPUT_CPMV_WIDTH, 'input signed', '\t\t\t// Motion vector 1. MSB = x component, LSB = y component.'],  
    ['INTERP_X', 1, 'output wire', '\t\t\t\t// Interpolate horizontally? 0: No. 1: Yes.'], 
    ['INTERP_Y', 1, 'output wire', '\t\t\t\t// Interpolate vertically? 0: No. 1: Yes.'], 
    ['CTRL_X', 1, 'output wire', '\t\t\t\t// {ctrl_X, ctrl_Y}? 00: Done all. 01: Next column. 10: Next line. 11: Next column.'], 
    ['CTRL_Y', 1, 'output wire', '\t\t\t\t// {ctrl_X, ctrl_Y}? 00: Done all. 01: Next column. 10: Next line. 11: Next column.'],
    ['OUT_GEN_MV_X_INTEGER', OUTPUT_MV_GEN_WIDTH-int(math.log2(INPUT_MACROBLOCK_SIZE)), 'output wire', ''],
    ['OUT_GEN_MV_Y_INTEGER', OUTPUT_MV_GEN_WIDTH-int(math.log2(INPUT_MACROBLOCK_SIZE)), 'output wire', ''],
    ['OUT_GEN_MV_X_FRAC', int(math.log2(INPUT_MACROBLOCK_SIZE)), 'output wire', ''],
    ['OUT_GEN_MV_Y_FRAC', int(math.log2(INPUT_MACROBLOCK_SIZE)), 'output wire', '']
    
]

# Intermediary signals 
# Template: [signal name, signal type, signal width, description]
# Obs: signal type must be in accordance with the component's signal type
MV_GEN_SIGNALS = [
    ########################### Registers ##########################
    ['OUT_REG_COORD_X', 'wire signed', INPUT_COORD_WIDTH, '\t\t\t// Max = 0111 1111 = 127'],
    ['OUT_REG_COORD_Y', 'wire signed', INPUT_COORD_WIDTH, '\t\t\t// Max = 0111 1111 = 127'],
    ['OUT_REG_X', 'wire signed', INPUT_COORD_WIDTH, '\t\t\t\t\t// Max = 0111 1111 = 127'],
    ['OUT_REG_Y', 'wire signed', INPUT_COORD_WIDTH, '\t\t\t\t\t// Max = 0111 1111 = 127'],
    ['OUT_REG_CPMV_0', 'wire signed', 2*INPUT_CPMV_WIDTH, '\t\t\t// MSB: horizontal, LSB: vertical'],
    ['OUT_REG_CPMV_1', 'wire signed', 2*INPUT_CPMV_WIDTH, '\t\t\t// MSB: horizontal, LSB: vertical'],
    ['OUT_REG_GEN_MV_X', 'wire signed', OUTPUT_MV_GEN_WIDTH, ''],
    ['OUT_REG_GEN_MV_Y', 'wire signed', OUTPUT_MV_GEN_WIDTH, ''],
    ['OUT_REG_COUNT_BLOCK', 'wire', int(math.log2(INPUT_MACROBLOCK_SIZE)), ''],

    ########################### Adders ##########################
    ['OUT_ADD_X', 'wire signed', INPUT_COORD_WIDTH, ''],
    ['OUT_ADD_Y', 'wire signed', INPUT_COORD_WIDTH, ''],
    ['OUT_ADD_COUNT_BLOCK', 'wire', 4, ''],

    ########################### Muxes ##########################
    ['OUT_MUX_X', 'wire signed', INPUT_COORD_WIDTH, ''],
    ['OUT_MUX_Y', 'wire signed', INPUT_COORD_WIDTH, ''],

    ########################### MV Gen ###########################
    ['OUT_MV_GEN_X', 'wire signed', OUTPUT_MV_GEN_WIDTH, ''],
    ['OUT_MV_GEN_Y', 'wire signed', OUTPUT_MV_GEN_WIDTH, '']

]

# Components informations 
# [component name, component type, [inputs names], [outputs], description] 
# Template: ['', '', [], , [], , ''],
MV_GEN_COMPONENTS = [
    ########################## Registers ##########################
    ['REG_COORD_X', 'reg_coord', ['RST_ASYNC_MV_GEN', 'WRITE_REGS_COORDS', 'COORD_X', '', '', ''],  
     ['OUT_REG_COORD_X', ''], 'Reg to store the X coord'],

    ['REG_COORD_Y', 'reg_coord', ['RST_ASYNC_MV_GEN', 'WRITE_REGS_COORDS', 'COORD_Y', '', '', ''], 
     ['OUT_REG_COORD_Y', ''], 'Reg to store the Y coord'],

    ['REG_X', 'reg_coord', ['RST_ASYNC_MV_GEN', 'WRITE_REG_X', 'OUT_MUX_X', '', '', ''], 
     ['OUT_REG_X', ''], 'Reg to store incremented X coord'],

    ['REG_Y', 'reg_coord', ['RST_ASYNC_MV_GEN', 'WRITE_REG_Y', 'OUT_MUX_Y', '', '', ''],
     ['OUT_REG_Y', ''], 'Reg to store incremented Y coord'],

    ['REG_CPMV_0', 'reg_MV', ['RST_ASYNC_MV_GEN', 'WRITE_REGS_CPMVS', 'CPMV_0', '', '', ''],  
     ['OUT_REG_CPMV_0', ''], 'Reg to store original CPMV_0'],

    ['REG_CPMV_1', 'reg_MV', ['RST_ASYNC_MV_GEN', 'WRITE_REGS_CPMVS', 'CPMV_1', '', '', ''],  
     ['OUT_REG_CPMV_1', ''], 'Reg to store original CPMV_1'],

    ['REG_GEN_MV_X', 'reg_gen_MV', ['RST_ASYNC_MV_GEN', 'WRITE_REGS_GEN_MVS', 'OUT_MV_GEN_X', '', '', ''], 
     ['OUT_REG_GEN_MV_X', ''], ''],

    ['REG_GEN_MV_Y', 'reg_gen_MV', ['RST_ASYNC_MV_GEN', 'WRITE_REGS_GEN_MVS', 'OUT_MV_GEN_Y', '', '', ''],
     ['OUT_REG_GEN_MV_Y', ''], ''],

    ['REG_COUNT_BLOCK', 'reg_count_block', ['RST_ASYNC_MV_GEN', 'WRITE_REG_COUNT_BLOCK', 'OUT_ADD_COUNT_BLOCK', '', '', ''], 
     ['OUT_REG_COUNT_BLOCK', ''], 'Reg to storE block-to-be-processed position'],

    ######################### Adders ##########################
    ['ADD_X', 'add_coords', [str(INPUT_COORD_WIDTH)+"'b1", 'OUT_REG_X', '', '', '', ''],
     ['OUT_ADD_X', ''], 'Adder to increment the X coord'],

    ['ADD_Y', 'add_coords', [str(INPUT_COORD_WIDTH)+"'b1", 'OUT_REG_Y', '', '', '', ''], 
     ['OUT_ADD_Y', ''], 'Adder to increment the Y coord'],

    ['ADD_COUNT_BLOCK', 'add_count_block', [str(int(math.log2(INPUT_MACROBLOCK_SIZE)))+"'b1", 'OUT_REG_COUNT_BLOCK', '', '', '', ''], 
     ['OUT_ADD_COUNT_BLOCK', ''], 'Adder to set the block to be processed'],

    ########################### Muxes ###########################
    ['MUX_X', 'mux_sel_coord', ['OUT_REG_COORD_X', 'OUT_ADD_X', 'SEL_X', '', '', ''],
     ['OUT_MUX_X', ''], ''],

    ['MUX_Y', 'mux_sel_coord', ['OUT_REG_COORD_Y', 'OUT_ADD_Y', 'SEL_Y', '', '', ''],
     ['OUT_MUX_Y', ''], ''],

    ########################### MV Geneator ###########################
    ['MVGEN4', 'MV_generator_4', ['OUT_REG_X', 'OUT_REG_Y', 'OUT_REG_CPMV_0[15:8]', 'OUT_REG_CPMV_1[15:8]', 'OUT_REG_CPMV_0[7:0]', 'OUT_REG_CPMV_1[7:0]'],
     ['OUT_MV_GEN_X', 'OUT_MV_GEN_Y'], 'Motion Vector generator of 4 parameters'] 
]

In [21]:
####################################################
# MV GENERATOR DATAPATH CALCULATIONS
####################################################

################## Inputs declaration ##################
IO_DECLARATION = []
for i in range(len(MV_GEN_CTRL_SIGNALS)):
    if MV_GEN_CTRL_SIGNALS[i][1] == 1:
        if i != len(MV_GEN_CTRL_SIGNALS)-1:
            IO_DECLARATION.append('  '+MV_GEN_CTRL_SIGNALS[i][2]+' '+MV_GEN_CTRL_SIGNALS[i][0]+','+MV_GEN_CTRL_SIGNALS[i][3])
        else:
            IO_DECLARATION.append('  '+MV_GEN_CTRL_SIGNALS[i][2]+' '+MV_GEN_CTRL_SIGNALS[i][0]+MV_GEN_CTRL_SIGNALS[i][3])
    else:
        if i != len(MV_GEN_CTRL_SIGNALS)-1:
            IO_DECLARATION.append('  '+MV_GEN_CTRL_SIGNALS[i][2]+' ['+str(MV_GEN_CTRL_SIGNALS[i][1]-1)+':0] '+MV_GEN_CTRL_SIGNALS[i][0]+','+MV_GEN_CTRL_SIGNALS[i][3])
        else:
            IO_DECLARATION.append('  '+MV_GEN_CTRL_SIGNALS[i][2]+' ['+str(MV_GEN_CTRL_SIGNALS[i][1]-1)+':0] '+MV_GEN_CTRL_SIGNALS[i][0]+MV_GEN_CTRL_SIGNALS[i][3])    
IO_DECLARATION_CONCAT = '\n'.join(IO_DECLARATION)


################## Signals declaration ##################
SIG_DECLARATION = []
for i in range(len(MV_GEN_SIGNALS)):
    SIG_DECLARATION.append('  '+MV_GEN_SIGNALS[i][1]+' ['+str(MV_GEN_SIGNALS[i][2]-1)+':0] '+MV_GEN_SIGNALS[i][0]+';'+MV_GEN_SIGNALS[i][3])
SIG_DECLARATION_CONCAT = '\n'.join(SIG_DECLARATION)


################## Components declarations ##################
COMPONENTS_DECLARATION = []
for i in range(len(MV_GEN_COMPONENTS)):
    if (MV_GEN_COMPONENTS[i][0][:3] == 'REG'):
        COMPONENTS_DECLARATION.append('  '+MV_GEN_COMPONENTS[i][1]+' '+MV_GEN_COMPONENTS[i][0]+' (\t\t//'+MV_GEN_COMPONENTS[i][4])
        COMPONENTS_DECLARATION.append('\t.CLK (CLK),')
        COMPONENTS_DECLARATION.append('\t.RST_ASYNC_N ('+MV_GEN_COMPONENTS[i][2][0]+'),')
        COMPONENTS_DECLARATION.append('\t.WRITE_EN ('+MV_GEN_COMPONENTS[i][2][1]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN ('+MV_GEN_COMPONENTS[i][2][2]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_OUT ('+MV_GEN_COMPONENTS[i][3][0]+'));\n')

    elif (MV_GEN_COMPONENTS[i][0][:3] == 'ADD'):
        COMPONENTS_DECLARATION.append('  '+MV_GEN_COMPONENTS[i][1]+' '+MV_GEN_COMPONENTS[i][0]+' (\t\t//'+MV_GEN_COMPONENTS[i][4])
        COMPONENTS_DECLARATION.append('\t.A ('+MV_GEN_COMPONENTS[i][2][0]+'),')
        COMPONENTS_DECLARATION.append('\t.B ('+MV_GEN_COMPONENTS[i][2][1]+'),')
        COMPONENTS_DECLARATION.append('\t.C ('+MV_GEN_COMPONENTS[i][3][0]+'));\n')

    elif (MV_GEN_COMPONENTS[i][0][:3] == 'MUX'):
        COMPONENTS_DECLARATION.append('  '+MV_GEN_COMPONENTS[i][1]+' '+MV_GEN_COMPONENTS[i][0]+' (\t\t//'+MV_GEN_COMPONENTS[i][4])
        COMPONENTS_DECLARATION.append('\t.DATA_IN_0 ('+MV_GEN_COMPONENTS[i][2][0]+'),')
        COMPONENTS_DECLARATION.append('\t.DATA_IN_1 ('+MV_GEN_COMPONENTS[i][2][1]+'),')
        COMPONENTS_DECLARATION.append('\t.SELECT ('+MV_GEN_COMPONENTS[i][2][2]+'),')        
        COMPONENTS_DECLARATION.append('\t.DATA_OUT ('+MV_GEN_COMPONENTS[i][3][0]+'));\n')   

    elif (MV_GEN_COMPONENTS[i][0][:3] == 'MVG'):
        COMPONENTS_DECLARATION.append('  '+MV_GEN_COMPONENTS[i][1]+' '+MV_GEN_COMPONENTS[i][0]+' (\t\t//'+MV_GEN_COMPONENTS[i][4])
        COMPONENTS_DECLARATION.append('\t.X_COORD ('+MV_GEN_COMPONENTS[i][2][0]+'),')
        COMPONENTS_DECLARATION.append('\t.Y_COORD ('+MV_GEN_COMPONENTS[i][2][1]+'),')
        COMPONENTS_DECLARATION.append('\t.MV_0_H ('+MV_GEN_COMPONENTS[i][2][2]+'),')
        COMPONENTS_DECLARATION.append('\t.MV_1_H ('+MV_GEN_COMPONENTS[i][2][3]+'),')
        COMPONENTS_DECLARATION.append('\t.MV_0_V ('+MV_GEN_COMPONENTS[i][2][4]+'),')
        COMPONENTS_DECLARATION.append('\t.MV_1_V ('+MV_GEN_COMPONENTS[i][2][5]+'),')
        COMPONENTS_DECLARATION.append('\t.MV_H_OUT ('+MV_GEN_COMPONENTS[i][3][0]+'),')
        COMPONENTS_DECLARATION.append('\t.MV_V_OUT ('+MV_GEN_COMPONENTS[i][3][1]+'));')

COMPONENTS_DECLARATION_CONCAT = '\n'.join(COMPONENTS_DECLARATION)


################## Combinational logic ##################
COMBINATIONAL_LOGIC = []
COMBINATIONAL_LOGIC.append("  assign OUT_GEN_MV_X_INTEGER = OUT_REG_GEN_MV_X["+str(OUTPUT_MV_GEN_WIDTH-1)+':'+str(int(math.log2(INPUT_MACROBLOCK_SIZE)))+'];')
COMBINATIONAL_LOGIC.append("  assign OUT_GEN_MV_Y_INTEGER = OUT_REG_GEN_MV_Y["+str(OUTPUT_MV_GEN_WIDTH-1)+':'+str(int(math.log2(INPUT_MACROBLOCK_SIZE)))+'];')
COMBINATIONAL_LOGIC.append("  assign OUT_GEN_MV_X_FRAC = OUT_REG_GEN_MV_X["+str(int(math.log2(INPUT_MACROBLOCK_SIZE))-1)+':0];')
COMBINATIONAL_LOGIC.append("  assign OUT_GEN_MV_Y_FRAC = OUT_REG_GEN_MV_Y["+str(int(math.log2(INPUT_MACROBLOCK_SIZE))-1)+':0];')
COMBINATIONAL_LOGIC.append("  assign INTERP_X = OUT_MV_GEN_X[0] | OUT_MV_GEN_X[1] | OUT_MV_GEN_X[2] | OUT_MV_GEN_X[3];")
COMBINATIONAL_LOGIC.append("  assign INTERP_Y = OUT_MV_GEN_X[0] | OUT_MV_GEN_X[1] | OUT_MV_GEN_X[2] | OUT_MV_GEN_X[3];")
COMBINATIONAL_LOGIC.append("  assign CTRL_X = OUT_REG_COUNT_BLOCK[2] ^ 1'b1 | OUT_REG_COUNT_BLOCK[3] ^ 1'b1;")
COMBINATIONAL_LOGIC.append("  assign CTRL_Y = OUT_REG_COUNT_BLOCK[0] ^ 1'b1 | OUT_REG_COUNT_BLOCK[1] ^ 1'b1;")

COMBINATIONAL_LOGIC_CONCAT = '\n'.join(COMBINATIONAL_LOGIC)


# print(IO_DECLARATION_CONCAT)
# print(SIG_DECLARATION_CONCAT)
# print(COMPONENTS_DECLARATION_CONCAT)
# print(COMBINATIONAL_LOGIC_CONCAT)

#### Datapath text 

In [22]:
#########################################
# Text writing: MV Gen datapath
#########################################

MV_GEN_DATAPATH_VERILOG = '''/*-----------------------------------------------------------------------------------
* File: '''+MV_GEN_FILE_NAME+'''.v
* Date generated: 25/03/2023
* Date modified: '''+DATE_MODIF+'''
* Author: Bruna Suemi Nagai
* Description: Motion Vector Generator datapath.
*----------------------------------------------------------------------------------- */

module '''+MV_GEN_FILE_NAME+''' (
'''+IO_DECLARATION_CONCAT+'''
);


// ------------------------------------------
// Signals declarations
// ------------------------------------------
'''+SIG_DECLARATION_CONCAT+'''


// ------------------------------------------
// Modules instantiation
// ------------------------------------------
'''+COMPONENTS_DECLARATION_CONCAT+'''


// ------------------------------------------
// Outputs
// ------------------------------------------
'''+COMBINATIONAL_LOGIC_CONCAT+'''


endmodule // '''+MV_GEN_FILE_NAME+'''
'''

# print(MV_GEN_DATAPATH_VERILOG)
with open('../verilog/datapaths/'+MV_GEN_FILE_NAME+'.v', 'w') as mv_datapath_file:
    mv_datapath_file.write(MV_GEN_DATAPATH_VERILOG)


In [ ]:
# MUX_SEL_BUF_IN_0 -> OUT_REG_INPUT_LINE[55:48] -> EXT_OUT_REG_INPUT_LINE_0
# MUX_SEL_BUF_IN_1 -> OUT_REG_INPUT_LINE[47:40] -> EXT_OUT_REG_INPUT_LINE_1
# MUX_SEL_BUF_IN_2 -> OUT_REG_INPUT_LINE[39:32] -> EXT_OUT_REG_INPUT_LINE_2
# MUX_SEL_BUF_IN_3 -> OUT_REG_INPUT_LINE[31:24] -> EXT_OUT_REG_INPUT_LINE_3



